# Roma Satellite Imagery Analysis with OpenEO

## Overview

This notebook demonstrates how to retrieve and visualize satellite imagery of Rome using the OpenEO API. We'll use Sentinel-2 data to create a true-color RGB image of the Rome area.

In [ ]:
# Import required packages
import openeo
from openeo.processes import process, array_create

## Connect to the OpenEO Backend

We connect to the OpenEO backend service and authenticate using OpenID Connect. This establishes a connection to the remote processing service where our satellite data is stored.

In [ ]:
# Connect to the back-end

# connection = openeo.connect(
#     "https://api.explorer.eopf.copernicus.eu/openeo"
# ).authenticate_oidc_authorization_code()
connection = openeo.connect(
    url="http://127.0.0.1:8081/"
).authenticate_oidc_authorization_code()

## Define Spatial Extent

We define the geographic boundaries of our area of interest using longitude and latitude coordinates.

In [ ]:
# Bretagne
spatial_extent_west = -5.0
spatial_extent_south = 40
spatial_extent_east = 5.0
spatial_extent_north = 50

## Load Sentinel-2 Data

We load Sentinel-2 satellite imagery for our defined area using the `load_collection` process. 
- We select the RGB bands (B04=Red, B03=Green, B02=Blue) for true-color visualization

In [ ]:
cube = connection.load_collection(
    "sentinel-2-l2a",
    spatial_extent={
        "west": spatial_extent_west,
        "south": spatial_extent_south,
        "east": spatial_extent_east,
        "north": spatial_extent_north,
    },
    temporal_extent=["2025-07-01", "2025-07-31"],
    bands=["reflectance|b04", "reflectance|b03", "reflectance|b02"],
)
cube

## Scale Image Values

Raw satellite reflectance values need to be scaled to standard RGB display range (0-255). We define a processing function that:
1. Scales the values from their original range (0-10000) to 0-255
2. Applies truncation to ensure all values are valid integers within range

In [ ]:
def _process1(x, context=None):
    data1 = process("linear_scale_range", inputMax=1, inputMin=0, outputMax=255, x=x)
    data2 = process("trunc", x=data1)
    return data2

## Apply Processing and Enhance Visual Appearance

We apply the scaling function to our data cube, then use a color formula to enhance the visual appearance of the image. The color formula applies:
- Gamma correction (RGB 1.5) to adjust brightness and contrast
- Sigmoidal contrast enhancement (RGB 10 0.3) to improve detail visibility
- Saturation adjustment to enhance color vibrancy

In [ ]:
# Reduce temporal dimension by taking the First value
reduced = cube.process(
    "apply_pixel_selection",
    pixel_selection="first",
    data=cube,
)


def _to_rgb(data):
    return array_create([data[0], data[1], data[2]])


rgb = reduced.apply_dimension(dimension="bands", process=_to_rgb)

processed = rgb.apply(process=_process1)
graph = processed.process(
    "color_formula",
    data=processed,
    formula="Gamma RGB 1.5 Sigmoidal RGB 6 0.3 Saturation 1",
)
graph = graph.save_result(format="PNG")

print(graph.to_json())

## Create a XYZ Tile Service


In [ ]:
# Create XYZ Service
service_extent = [
    spatial_extent_west,
    spatial_extent_south,
    spatial_extent_east,
    spatial_extent_north,
]

service = connection.create_service(
    graph,
    type="XYZ",
    title="Sentinel-2 L2A RGB - July 2025 - Bretagne",
    description="Sentinel-2 L2A RGB composite for July 2025 over Bretagne",
    configuration={
        "tile_size": 256,
        "minzoom": 5,
        "maxzoom": 14,
        "extent": service_extent,
    },
)
print(service)

In [ ]:
from folium import Map, TileLayer

m = Map(
    location=(
        (service_extent[3] + service_extent[1]) / 2,
        (service_extent[0] + service_extent[2]) / 2,
    ),
    zoom_start=8,
)

# backend = "https://api.explorer.eopf.copernicus.eu/openeo"
backend = "http://127.0.0.1:8181"

TileLayer(
    tiles=f"{backend}/services/xyz/{service.service_id}/tiles/{{z}}/{{x}}/{{y}}",
    opacity=1,
    attr="EOPF",
).add_to(m)
m

## Conclusion

This notebook demonstrates how to use OpenEO to access, process, and visualize satellite imagery for urban areas. The workflow can be extended to include additional analysis, such as:

- Time series analysis to observe changes over multiple dates
- Land cover classification to identify different urban features
- Spectral indices to analyze vegetation, water, or built-up areas
- Image segmentation to extract specific features